In [2]:
import jijmodeling as jm

def build_mis_unconstrained() -> jm.Problem:
    """Create unconstrained Maximum Independent Set (MIS) model in QUBO form.

    Formulates the MIS problem as a quadratic unconstrained binary optimization
    (QUBO), where the objective maximizes the number of selected vertices while
    penalizing the selection of adjacent vertices.

    Objective:
        maximize  Σ_v x[v]  −  2 Σ_(u,v)∈E x[u]·x[v]

    Assumptions:
        - Edge list E must contain each undirected edge only once.
        - No self-loops are allowed.

    Returns:
        jm.Problem: JijModeling problem instance with objective defined in
        quadratic unconstrained form.
    """
    # -------- Placeholders --------
    N = jm.Placeholder("N", description="number of nodes")
    # E is a (num_edges, 2) integer array (0-based endpoints)
    E = jm.Placeholder("E", ndim=2, description="edge list as pairs (u,v), 0-based")

    # -------- Decision vars --------
    x = jm.BinaryVar("x", shape=(N,), description="1 if vertex i is chosen")

    # -------- Objective (quadratic) --------
    v = jm.Element("v", belong_to=(0, N))
    e = jm.Element("e", belong_to=E)

    obj = jm.sum(v, x[v]) - 2 * jm.sum(e, x[e[0]] * x[e[1]])

    # -------- Problem (no constraints) --------
    prob = jm.Problem("mis_unconstrained_qubo", sense=jm.ProblemSense.MAXIMIZE)
    prob += obj

    return prob


In [ ]:
def read_dimacs_gph(path: str):
    """Read an undirected graph from a DIMACS .gph file.

    Supports the DIMACS graph format with the following conventions:
        - Lines starting with `c` are comments.
        - A header line of the form `p edge N M` specifies the number of
          vertices (N) and edges (M).
        - Each edge is given by a line `e u v` with 1-based vertex indices.

    Args:
        path (str): Path to the DIMACS .gph file.

    Returns:
        tuple:
            - int: Number of vertices `N`.
            - list[list[int]]: Edge list `E`, where each element is `[u, v]`
              with 0-based endpoints.

    Example:
        >>> N, E = read_dimacs_gph("example.gph")
        >>> N
        5
        >>> E
        [[0, 1], [0, 2], [1, 3]]
    """
    N = None
    E = []
    with open(path, "r") as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith("c"):
                continue
            if line.startswith("p"):
                # e.g. "p edge 17 39"
                parts = line.split()
                if len(parts) < 4 or parts[1] != "edge":
                    raise ValueError(f"Invalid p-line: {line}")
                N = int(parts[2])
                # M = int(parts[3])  # Can be used for validation if needed
            elif line.startswith("e"):
                # e.g. "e 7 17"
                _, u, v = line.split()
                u0 = int(u) - 1   # convert to 0-based
                v0 = int(v) - 1
                if u0 == v0:
                    # skip self-loops
                    continue
                if v0 < u0:
                    u0, v0 = v0, u0
                E.append([u0, v0])

    if N is None:
        raise ValueError("Missing 'p edge N M' header.")
    # Deduplicate and normalize
    E = sorted(set(tuple(e) for e in E))
    E = [list(e) for e in E]

    if any(u < 0 or v < 0 or u >= N or v >= N for u, v in E):
        raise ValueError("Edge endpoint out of range for declared N.")

    return N, E


In [ ]:
import ommx
problem_unconstrained = build_mis_unconstrained()

In [5]:
N, E = read_dimacs_gph("farm.gph")
instance_data = {"N": N, "E": E}
ommx.v1.Instance = jm.Interpreter(instance_data).eval_problem(problem_unconstrained)

In [6]:
ommx.v1.Instance.objective

Function(-2*x0*x5 - 2*x0*x6 - 2*x0*x7 - 2*x0*x8 - 2*x0*x9 - 2*x0*x10 - 2*x0*x11 - 2*x0*x12 - 2*x0*x13 - 2*x0*x14 - 2*x1*x5 - 2*x1*x6 - 2*x1*x7 - 2*x1*x8 - 2*x1*x9 - 2*x1*x10 - 2*x1*x11 - 2*x1*x12 - 2*x1*x13 - 2*x1*x14 - 2*x2*x5 - 2*x2*x6 - 2*x2*x11 - 2*x2*x13 - 2*x2*x15 - 2*x3*x5 - 2*x3*x6 - 2*x3*x8 - 2*x3*x10 - 2*x3*x12 - 2*x3*x14 - 2*x3*x16 - 2*x4*x6 - 2*x4*x9 - 2*x5*x11 - 2*x5*x12 - 2*x6*x13 - 2*x6*x14 - 2*x6*x16 + x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14 + x15 + x16)

In [7]:
import ommx_pyscipopt_adapter as scip_ad
solution = scip_ad.OMMXPySCIPOptAdapter.solve(ommx.v1.Instance)
print(f"{solution.objective=}, {solution.feasible=}")

solution.objective=10.0, solution.feasible=True


In [8]:
solution.decision_variables_df

,kind,lower,upper,name,subscripts,description,substituted_value,value
id,,,,,,,,
0,Binary,0.0,1.0,x,[0],<NA>,<NA>,0.0
1,Binary,0.0,1.0,x,[1],<NA>,<NA>,0.0
2,Binary,0.0,1.0,x,[2],<NA>,<NA>,0.0
3,Binary,0.0,1.0,x,[3],<NA>,<NA>,0.0
4,Binary,0.0,1.0,x,[4],<NA>,<NA>,0.0
5,Binary,0.0,1.0,x,[5],<NA>,<NA>,0.0
6,Binary,0.0,1.0,x,[6],<NA>,<NA>,0.0
7,Binary,0.0,1.0,x,[7],<NA>,<NA>,1.0
8,Binary,0.0,1.0,x,[8],<NA>,<NA>,1.0
